In [73]:
import requests
import os
import json
import datetime
import time

api_key = ""
url = "https://www.virustotal.com/api/v3/urls"

payload = { "url": "www.rifsltd.co.uk" }
headers = {
    "accept": "application/json",
    "x-apikey": api_key,
    "content-type": "application/x-www-form-urlencoded"
}

In [74]:
# first we have to go and get the request for the url 
response = requests.post(url, data=payload, headers=headers)
response_json = json.loads(response.text)

# from the response we can get the link to the actual scan
scan_response = requests.get(response_json['data']['links']['self'], headers=headers)
scan_response_json = json.loads(scan_response.text)
dt = datetime.datetime.fromtimestamp(scan_response_json['data']['attributes']['date']).strftime("%d/%m/%Y %H:%M:%S")
while scan_response_json['data']['attributes']['status'] != 'completed':
    print(f"Scan status : {scan_response_json['data']['attributes']['status']}")
    time.sleep(30)
    scan_response = requests.get(response_json['data']['links']['self'], headers=headers)
    scan_response_json = json.loads(scan_response.text)
    
scan_results = scan_response_json['data']['attributes']['stats']

Scan status : queued


In [75]:
print(scan_response_json['data']['attributes']['status'])
print(f"{dt}: {scan_results}")

completed
03/06/2024 20:36:05: {'malicious': 0, 'suspicious': 0, 'undetected': 27, 'harmless': 68, 'timeout': 0}


In [76]:
def decision_engine(scan_results):
    total = scan_results['malicious'] + scan_results['suspicious'] + scan_results['harmless']
    bad = scan_results['malicious'] + scan_results['suspicious']
    harmless = scan_results['harmless']
    percentage = round(harmless / total * 100)
    print(percentage)
    if percentage >= 5:
        print(f"PASSED : {percentage}% confirmed ok")
    else:
        print(f"FAILED threshold! : {bad} engines report suspicious or malicious. Out of {total}. Refer for manual checking")

In [77]:
decision_engine(scan_results)

100
PASSED : 100% confirmed ok


In [17]:
scan_response_json['data']['attributes']['date']

1717436234

In [22]:
dt = datetime.datetime.fromtimestamp(scan_response_json['data']['attributes']['date'])
dt.strftime("%d/%m/%Y %H:%M:%S")

'03/06/2024 18:37:14'

In [23]:
scan_response_json['data']['attributes']['stats']

{'malicious': 1,
 'suspicious': 1,
 'undetected': 26,
 'harmless': 67,
 'timeout': 0}

In [24]:
type(scan_response_json['data']['attributes']['stats'])

dict